In [1]:
import torch
import torchvision
import torch.optim as optim
from tqdm import tqdm

import torchvision.transforms as transforms
from helpers import *
from networks import *



In [2]:
# Define the transformations for the CIFAR10, ImageNet, MNIST datasets

cifar10_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

imagenet_train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

imagenet_test_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

mnist_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

## CIFAR 10

In [3]:
# load CIFAR10 dataset
train_loader,test_loader = load_dataset(dataset_name='CIFAR10',batch_size = 100, train_transform=cifar10_transform,
                                        test_transform= cifar10_transform, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# build network
net = CNN1()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
def train_model(train_loader,network ,optimizer, criterion, max_iter= 1000, model_name = ''):
    losses = []
    for epoch in tqdm(range(max_iter)):
        running_loss = 0.0
        for _,data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        losses.append(running_loss)
        running_loss = 0.0
    PATH = os.path.join('models','{}_network.pth'.format(model_name))
    torch.save(network.state_dict(), PATH)
    return losses


In [7]:
losses = train_model(train_loader,net,optimizer,criterion,max_iter = 10,model_name='CIFAR10')

100%|██████████| 10/10 [02:50<00:00, 17.00s/it]


In [8]:
losses

[699.7059220075607,
 685.0095592737198,
 671.2777829170227,
 658.0299317836761,
 645.6872897744179,
 635.1241436600685,
 623.0764491558075,
 611.8926228880882,
 601.8196529746056,
 592.2796945571899]

In [10]:
def test_model(test_loader,network):
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for _,data in enumerate(test_loader):
            images, labels = data
            outputs = network(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_predictions += 1
                total_predictions += 1
    accuracy = correct_predictions/total_predictions
    return accuracy

In [12]:
accuracy = test_model(test_loader,net)
accuracy

0.5655